In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["USER"] = "root" # TODO change this to your username

In [2]:
# import libs
import torch
import torchaudio
import numpy as np
import random
from argparse import Namespace

from data.tokenizer import (
    AudioTokenizer,
    TextTokenizer,
)

from models import voicecraft

/work/anaconda/envs/dhy_voicecraft/lib/python3.9/site-packages/phonemizer/utils.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
!export HF_ENDPOINT=https://hf-mirror.com

In [ ]:
# hyperparameters for inference
left_margin = 0.08
right_margin = 0.08
codec_audio_sr = 16000
codec_sr = 50
top_k = 40
top_p = 1
temperature = 1
kvcache = 0
# adjust the below three arguments if the generation is not as good
seed = 1 # random seed magic
silence_tokens = [1388,1898,131] # if there are long silence in the generated audio, reduce the stop_repetition to 3, 2 or even 1
stop_repetition = -1 # -1 means do not adjust prob of silence tokens. if there are long silence or unnaturally strecthed words, increase sample_batch_size to 2, 3 or even 4
# what this will do to the model is that the model will run sample_batch_size examples of the same audio, and pick the one that's the shortest
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_everything(seed)
device = "cuda" if torch.cuda.is_available() else "cpu"
# load model, tokenizer, and other necessary files
voicecraft_name="giga830M.pth" # or gigaHalfLibri330M_TTSEnhanced_max16s.pth, giga830M.pth

# the new way of loading the model, with huggingface, recommended
# from models import voicecraft
# phn2num = model.args.phn2num
# config = vars(model.args)
# model.to(device)

# # the old way of loading the model
from models import voicecraft
from huggingface_hub import hf_hub_download
model_dir = "/tsdata2/dhy/models/VoiceCraft"
# filepath = hf_hub_download(repo_id="pyp1/VoiceCraft", filename=voicecraft_name, repo_type="model")
filepath = f"{model_dir}/{voicecraft_name}"
ckpt = torch.load(filepath, map_location="cpu")
model = voicecraft.VoiceCraft(ckpt["config"])
model.load_state_dict(ckpt["model"])
config = vars(model.args)
phn2num = ckpt["phn2num"]
model.to(device)
model.eval()

encodec_fn = f"{model_dir}/encodec_4cb2048_giga.th"
if not os.path.exists(encodec_fn):
    os.system(f"wget https://huggingface.co/pyp1/VoiceCraft/resolve/main/encodec_4cb2048_giga.th")
    os.system(f"mv encodec_4cb2048_giga.th ./pretrained_models/encodec_4cb2048_giga.th")
audio_tokenizer = AudioTokenizer(signature=encodec_fn) # will also put the neural codec model on gpu

text_tokenizer = TextTokenizer(backend="espeak")

# point to the original file or record the file
# write down the transcript for the file, or run whisper to get the transcript (and you can modify it if it's not accurate), save it as a .txt file
orig_audio = "./demo/84_121550_000074_000000.wav"
orig_transcript = "But when I had approached so near to them The common object, which the sense deceives, Lost not by distance any of its marks,"
# move the audio and transcript to temp folder
temp_folder = "./demo/temp"
os.makedirs(temp_folder, exist_ok=True)
os.system(f"cp {orig_audio} {temp_folder}")
filename = os.path.splitext(orig_audio.split("/")[-1])[0]
with open(f"{temp_folder}/{filename}.txt", "w") as f:
    f.write(orig_transcript)
# run MFA to get the alignment
align_temp = f"{temp_folder}/mfa_alignments"
dict_path = "/tsdata2/dhy/models/mfa_aligner/dictionary/english_us_arpa.dict"
os.makedirs(align_temp, exist_ok=True)
os.system(f"mfa align -j 1 --output_format csv {temp_folder} {dict_path} english_us_arpa {align_temp}")
# if it fail, it could be because the audio is too hard for the alignment model, increasing the beam size usually solves the issue
# os.system(f"mfa align -j 1 --output_format csv {temp_folder} english_us_arpa english_us_arpa {align_temp} --beam 1000 --retry_beam 2000")
audio_fn = f"{temp_folder}/{filename}.wav"
transcript_fn = f"{temp_folder}/{filename}.txt"
align_fn = f"{align_temp}/{filename}.csv"



/tmp/ipykernel_42951/2461702271.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(filepath, map_location="cpu")
/tsdata2/dhy/tse/VoiceCraft/src/audiocra

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1  [ 0:00:00 < 0:00:00 , ? it/s ]


In [ ]:
editTypes_set = set(['substitution', 'insertion', 'deletion'])
# propose what do you want the target modified transcript to be
target_transcript = "But when I saw the mirage of the lake in the distance, which the sense deceives, Lost not by distance any of its marks,"
edit_type = "substitution"
assert edit_type in editTypes_set, f"Invalid edit type {edit_type}. Must be one of {editTypes_set}."

# if you want to do a second modification on top of the first one, write down the second modification (target_transcript2, type_of_modification2)
# make sure the two modification do not overlap, if they do, you need to combine them into one modification

# run the script to turn user input to the format that the model can take
from edit_utils import get_span
orig_span, new_span = get_span(orig_transcript, target_transcript, edit_type)
print(f"orig_span: {orig_span}, new_span: {new_span}")
if orig_span[0] > orig_span[1]:
    RuntimeError(f"example {audio_fn} failed")
if orig_span[0] == orig_span[1]:
    orig_span_save = [orig_span[0]]
else:
    orig_span_save = orig_span
if new_span[0] == new_span[1]:
    new_span_save = [new_span[0]]
else:
    new_span_save = new_span

orig_span_save = ",".join([str(item) for item in orig_span_save])
new_span_save = ",".join([str(item) for item in new_span_save])
print(f"orig_span_save: {orig_span_save}, new_span_save: {new_span_save}")
from inference_speech_editing_scale import get_mask_interval

start, end = get_mask_interval(align_fn, orig_span_save, edit_type)
print(f"start, end: {start}s, {end}s")
info = torchaudio.info(audio_fn)
audio_dur = info.num_frames / info.sample_rate
morphed_span = (max(start - left_margin, 1/codec_sr), min(end + right_margin, audio_dur)) # in seconds

# span in codec frames
mask_interval = [[round(morphed_span[0]*codec_sr), round(morphed_span[1]*codec_sr)]]
mask_interval = torch.LongTensor(mask_interval) # [M,2], M==1 for now



# run the model to get the output
from inference_speech_editing_scale import inference_one_sample

decode_config = {'top_k': top_k, 'top_p': top_p, 'temperature': temperature, 'stop_repetition': stop_repetition, 'kvcache': kvcache, "codec_audio_sr": codec_audio_sr, "codec_sr": codec_sr, "silence_tokens": silence_tokens}
orig_audio, new_audio = inference_one_sample(model, Namespace(**config), phn2num, text_tokenizer, audio_tokenizer, audio_fn, target_transcript, mask_interval, device, decode_config)
        
# save segments for comparison
orig_audio, new_audio = orig_audio[0].cpu(), new_audio[0].cpu()
# logging.info(f"length of the resynthesize orig audio: {orig_audio.shape}")

# display the audio
from IPython.display import Audio
print("original:")
display(Audio(orig_audio, rate=codec_audio_sr))

print("edited:")
display(Audio(new_audio, rate=codec_audio_sr))

# save the audio
# output_dir
output_dir = "./demo/generated_se"
os.makedirs(output_dir, exist_ok=True)

save_fn_new = f"{output_dir}/{os.path.basename(audio_fn)[:-4]}_new_seed{seed}.wav"

torchaudio.save(save_fn_new, new_audio, codec_audio_sr)

save_fn_orig = f"{output_dir}/{os.path.basename(audio_fn)[:-4]}_orig.wav"
if not os.path.isfile(save_fn_orig):
    orig_audio, orig_sr = torchaudio.load(audio_fn)
    if orig_sr != codec_audio_sr:
        orig_audio = torchaudio.transforms.Resample(orig_sr, codec_audio_sr)(orig_audio)
    torchaudio.save(save_fn_orig, orig_audio, codec_audio_sr)

# # if you get error importing T5 in transformers
# # try 
# # pip uninstall Pillow
# # pip install Pillow
# # you are likely to get warning looks like WARNING:phonemizer:words count mismatch on 300.0% of the lines (3/1), this can be safely ignored

orig_span: [3, 11], new_span: [3, 11]
orig_span_save: 3,11, new_span_save: 3,11
start, end: 0.46, 3.6
original:


edited:


In [1]:
from edit_utils import get_span
orig_span, new_span = get_span("Thus was she borne away captive of her dead, neither willing nor unwilling, of life and death equally careless.", 
                               "Thus was she borne away of life and death equally careless.", "deletion")
print(f"orig_span: {orig_span}, new_span: {new_span}")


orig_span: [5, 12], new_span: [4, 5]
